In [1]:
# !pip install openai
# !pip install numpy
# !pip install quippy-ase
# !pip install python-dotenv

In [2]:
from main import *
import importlib

In [3]:
messages = [
    {
    "role": "system",
    "content": "You are an expert computational materials scientist. \nYou are trying to develop the EXACT experimental conditions used to crystallise carbon into diamond.\nPredict appropriate code for a melt-quench simulation of this system. \nTry NOT to use an extremely large number of timesteps.\nThe format of the code should be a LAMMPS input file like:\n# Phase PHASE:\nfix integrate all nvt temp T_INITIAL T_FINAL 0.1\nrun TIMESTEPS\nunfix integrate\n"
    },
    {
    "role": "user",
    "content": "RESULT:\nSOAP similarity to diamond is 0.1\nSOAP similarity to graphite is 0.9\n\nSCRIPT:"
    },
    {
    "role": "assistant",
    "content": "# Phase 1: \nfix integrate all nvt temp 9000 9000 0.1\nrun 100000\nunfix integrate\n\n# Phase 2: \nfix integrate all nvt temp 9000 3000 0.1\nrun 100000 \nunfix integrate\n\n# Phase 3: \nfix integrate all nvt temp 3000 500 0.1\nrun 100000 \nunfix integrate\n\n# Phase 4:\nfix integrate all nvt temp 500 500 0.1\nrun 100000 \nunfix integrate\n\n# Phase 5:\nfix integrate all nvt temp 500 300 0.1\nrun 100000\nunfix integrate"
    },
    {
    "role": "user",
    "content": "RESULT:\nSOAP similarity to diamond is 1.0\nSOAP similarity to graphite is 0.0\n\nSCRIPT:"
    },
]


In [4]:
code, messages, response = query_llm(messages)

In [5]:
code # this part made by llm  
# \nfix integrate all nvt temp 5000 5000 0.1\nrun 50000\nunfix integrate\n\n# Phase 2: \nfix integrate all nvt temp 5000 3000 0.1\nrun 50000 \nunfix integrate\n\n# Phase 3: \nfix integrate all nvt temp 3000 1000 0.1\nrun 50000 \nunfix integrate\n\n# Phase 4:\nfix integrate all nvt temp 1000 1000 0.1\nrun 50000 \nunfix integrate\n\n# Phase 5:\nfix integrate all nvt temp 1000 300 0.1\nrun 50000\nunfix integrate

'# Phase 1: \nfix integrate all nvt temp 5000 5000 0.1\nrun 50000\nunfix integrate\n\n# Phase 2: \nfix integrate all nvt temp 5000 3000 0.1\nrun 50000 \nunfix integrate\n\n# Phase 3: \nfix integrate all nvt temp 3000 1000 0.1\nrun 50000 \nunfix integrate\n\n# Phase 4:\nfix integrate all nvt temp 1000 1000 0.1\nrun 50000 \nunfix integrate\n\n# Phase 5:\nfix integrate all nvt temp 1000 300 0.1\nrun 50000\nunfix integrate'

In [6]:
s, rundir = format_experiment()
s, rundir 

formatting experiment for rundir:
 /home/epsilon/vld/univ5120/VLD/Thesis/llms-for-lammps/run/test_run


('\nlog /home/epsilon/vld/univ5120/VLD/Thesis/llms-for-lammps/run/test_run/log.dat append\n\nunits metal     # mass = g/mole, distance = Å, time = ps, energy = eV, force = eV/Å\natom_style atomic\nread_data /home/epsilon/vld/univ5120/VLD/Thesis/llms-for-lammps/structures/example.data\nmass * 12.011   # all atoms are Carbon and therefore have mass ~12\n\npair_style quip\npair_coeff * * /home/epsilon/vld/univ5120/VLD/Thesis/llms-for-lammps/data/carbon.xml "" 6\n\nneigh_modify every 1 delay 0 check yes\ntimestep 0.001\n\nvariable nAtoms equal atoms\n\nfix removeMomentum all momentum 1 linear 1 1 1\n\ncompute T all temp\nvariable P equal press\nvariable v equal vol\nvariable PE_Atom equal c_PE/v_nAtoms\n\nfix TempAve all ave/time 100 1.0 100 c_T\nfix PressAve all ave/time 100 1.0 100 v_P\nfix vAve all ave/time 100 1.0 100 v_v\nfix PEAve_Atom all ave/time 100 1.0 100 v_PE_Atom\n\ncompute PE all pe pair\ncompute MSD all msd\n\nthermo_style custom step cpu temp f_TempAve press f_PressAve f_PE

In [7]:
full_code = s + code
# f_in="/home/jbutch/Projects/Thesis/llms-for-lammps/lammps/test.in"
f_in='/u/vld/univ5120/VLD/Thesis/llms-for-lammps/lammps/test.in'
with open(f_in, "w") as f:
    f.write(full_code)

print(full_code)


log /home/epsilon/vld/univ5120/VLD/Thesis/llms-for-lammps/run/test_run/log.dat append

units metal     # mass = g/mole, distance = Å, time = ps, energy = eV, force = eV/Å
atom_style atomic
read_data /home/epsilon/vld/univ5120/VLD/Thesis/llms-for-lammps/structures/example.data
mass * 12.011   # all atoms are Carbon and therefore have mass ~12

pair_style quip
pair_coeff * * /home/epsilon/vld/univ5120/VLD/Thesis/llms-for-lammps/data/carbon.xml "" 6

neigh_modify every 1 delay 0 check yes
timestep 0.001

variable nAtoms equal atoms

fix removeMomentum all momentum 1 linear 1 1 1

compute T all temp
variable P equal press
variable v equal vol
variable PE_Atom equal c_PE/v_nAtoms

fix TempAve all ave/time 100 1.0 100 c_T
fix PressAve all ave/time 100 1.0 100 v_P
fix vAve all ave/time 100 1.0 100 v_v
fix PEAve_Atom all ave/time 100 1.0 100 v_PE_Atom

compute PE all pe pair
compute MSD all msd

thermo_style custom step cpu temp f_TempAve press f_PressAve f_PEAve_Atom vol f_vAve c_MSD[4]
ther

In [2]:
import utils
from ase.io import read
import sys
import importlib; importlib.reload(utils)

# process dump files .dat to .xyz
# f='/u/vld/univ5120/VLD/Thesis/llms-for-lammps/run/test_run/dump_file.dump.50000.dat' # 0.996
# f='/u/vld/univ5120/VLD/Thesis/llms-for-lammps/run/test_run/dump_file.dump.100000.dat'  # 0.999
f='/u/vld/univ5120/VLD/Thesis/llms-for-lammps/run/test_run/dump_file.dump.250000.dat'  # 0.9901
atoms = read(f)
print(atoms.get_atomic_numbers())
atoms.set_atomic_numbers([6 for i in range(len(atoms))])
x = utils.analyse(atoms)
atoms, x

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


(Atoms(symbols='C200', pbc=True, cell=[15.854535682762254, 15.854535682762254, 15.854535682762254]),
 ({'graphite': 0.1768, 'diamond': 0.9408, 'TARGET': 0.9901},
  'RESULT:\nSOAP Similarity to graphite is 0.1768 \nSOAP Similarity to diamond is 0.9408 \nSOAP Similarity to TARGET is 0.9901 \nSCRIPT:'))

In [31]:
from dscribe.descriptors import SOAP
from ase.io import read, write
from ase import build
import numpy as np
dia_at = build.bulk('C', 'diamond', a=3.567)
gra_at = build.graphene('C2', a=2.46)
write('structures/diamond.xyz', dia_at)
write('structures/graphite.xyz', gra_at)

In [24]:
dia_supercell = dia_at * (2, 2, 2)
gra_supercell = gra_at * (3, 3, 1)

# Define the magnitude of random perturbation
perturbation_magnitude = 0.001  # e.g., 0.01 Angstrom

# Apply random perturbations to atomic positions
dia_supercell.positions += (np.random.rand(*dia_supercell.positions.shape)) * perturbation_magnitude
gra_supercell.positions += (np.random.rand(*gra_supercell.positions.shape)) * perturbation_magnitude

# Save the structures to XYZ files
write('structures/diamond_supercell.xyz', dia_supercell)
write('structures/graphite_supercell.xyz', gra_supercell)